In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import geopandas
import pandas
import pathlib
import numpy
import matplotlib
import seaborn
import shapely
import rioxarray
import matplotlib.pyplot
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# Plan
* Load in GeoFabrics estimated width, depth and water surface height - (using those used in GeoFabrics - i.e. same level of smoothing)
  * Calculate channel area - assume rectangular
* Load in measured cross sections
  * Transform to NZVD2016
  * Calculate the channel area - use the same water height as the GeoFabrics estimates
* Compare the results
  * Take the nearest GeoFabrics cross section to each measured cross section
  * Plot area's on the same plot

In [ ]:
resolution = 8
horizontal_crs = 2193

### Load in GeoFabrics Bathymetry estimates

In [ ]:
cache_path = pathlib.Path(r"C:\Local\data\catchments\Westport\caches\NZ20_Westport")
folder = "paper"
dems = {}
dems["Unconditioned"] = rioxarray.rioxarray.open_rasterio(cache_path / folder  / "geofabric_8m_unconditioned.nc", masked=True).squeeze("band", drop=True)
dems["Conceptual Multivariate Regression Approach"] = rioxarray.rioxarray.open_rasterio(cache_path / folder  / "geofabric_8m_rupp_and_smart.nc", masked=True).squeeze("band", drop=True)
dems["Uniform Flow Approach"] = rioxarray.rioxarray.open_rasterio(cache_path / folder  / "geofabric_8m_neal_et_al.nc", masked=True).squeeze("band", drop=True)
dems["Interpolated Surveyed Elevation Cross Sections"] = rioxarray.rioxarray.open_rasterio(cache_path / folder / "geofabric_8m_measured_river.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 1"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_1_area_change.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 2"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_2_variable_thresh.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 3"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_3_no_area_adjustment.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 4"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_4_area_adjustment.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 5"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_5_max_bank_height.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 6"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_6_to_ocean.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 7"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_7_fix_stop.nc", masked=True).squeeze("band", drop=True)
#dems["Test Rupp 8"] = rioxarray.rioxarray.open_rasterio(cache_path / "test" / "dem_8m_rupp_wide_8_max_bank_7m.nc", masked=True).squeeze("band", drop=True)


In [ ]:
# River polygon and centreline from the measured sections
polygon = geopandas.read_file(cache_path / folder / "measured_river" / "river_polygon.geojson")
riverbanks = geopandas.read_file(pathlib.Path(r"C:\Local\data\catchments") / "measured_sections" / "westport" / "riverbanks.geojson")
normalised_distances = numpy.arange(0, riverbanks.length.max())/riverbanks.length.max()
riverbanks=riverbanks.apply(lambda row: row.geometry.interpolate(normalised_distances, normalized=True), axis=1)
river_centreline = geopandas.GeoDataFrame(geometry=[shapely.geometry.LineString([shapely.geometry.MultiPoint([right, left]).centroid for (right, left) in zip(riverbanks.loc[0], riverbanks.loc[1])])],
                                          crs=horizontal_crs)

## Load in Measured cross sections
See raw cell below for converting to NZVD2016. This has already been done so load in the converted one directly

In [ ]:
sections_path = pathlib.Path(r"C:\Local\data\river_bathemetry\sections\Westport\3738_Buller_River_October_2021")
cross_section_elevation_label = 'Elevation'
cross_section_name_label = 'Name'
measured_sections = geopandas.read_file(sections_path / "3738_Buller_River_October_2021_NZVD2016.geojson")
all_measured_sections = ['Section1', 'Section2', 'Section2A', 'Section2B', 'Section3', 'Section3A', 'Section4', 'Section5', 'Section6',
                             'Section7', 'Section8', 'Section9', 'Section10', 'Section11', 'Section12', 'Section13', 'Section14',
                             'Section15', 'Section16', 'Section21', 'Section22', 'Section23', 'Section24', 'Section25', 'Section26', 'Section17',
                             'Section18', 'Section19', 'Section20']
ordered_measured_sections = ['Section1', 'Section2', 'Section2A', 'Section2B', 'Section3', 'Section3A', 'Section4', 'Section5', 'Section6',
                             'Section7', 'Section8', 'Section9', 'Section10', 'Section11', 'Section12', 'Section13', 'Section14',
                             'Section15', 'Section16', 'Section21', 'Section22', 'Section23', 'Section24', 'Section25', 'Section26']

In [ ]:
measured_sections.head(1)

### Simplify the measured points to min point/line strings

In [ ]:
measured_centres = {'Section': [], 'geometry': []}
measured_lines = {'Section': [], 'geometry': [], 'min_elevation': []}
for section in measured_sections[cross_section_name_label].unique():
    measured_section = measured_sections[measured_sections[cross_section_name_label] == section]
    measured_centre = measured_section[measured_section[cross_section_elevation_label]==measured_section[cross_section_elevation_label].min()].iloc[0]
    measured_centres['Section'].append(section)
    measured_centres['geometry'].append(measured_centre['geometry'])
    
    measured_lines['Section'].append(section)
    measured_lines['geometry'].append(shapely.geometry.LineString( [[measure_point.x, measure_point.y, measure_point.z] for measure_point in measured_section.geometry.values] ))
    measured_lines['min_elevation'].append(measured_section[cross_section_elevation_label].min())
measured_centres = geopandas.GeoDataFrame(measured_centres, crs=horizontal_crs)
measured_lines = geopandas.GeoDataFrame(measured_lines, crs=horizontal_crs)

## Sample DEM for elevations
Record the Bias and STDev for each cross section against the two DEMS

In [ ]:
import matplotlib.pyplot
fig, ax = matplotlib.pyplot.subplots(ncols=1, nrows=1, figsize=(10, 5))
polygon.plot(ax=ax, color='g')
measured_lines.plot(ax=ax)

# Sample every 1m
Use Linear interpolation in between

In [ ]:
comparisons_sampled = {**dict((f"{key} z", []) for key in dems), 
                       **{"Surveyed Cross Section Elevations z": [], "Section": [], "River": [], "Distance Upstream [m]": [], "Offset [m]": []}}
for index, row in measured_lines.iterrows():
    print(f"Section number = {index}")
    line = row.geometry
    distance_upstream = float(river_centreline.project(measured_centres.iloc[index].geometry))
    for offset in list(range(int(numpy.ceil(line.length)))):
        point = line.interpolate(offset)
        for key in dems:
            comparisons_sampled[f"{key} z"].append(float(dems[key].z.interp(x=point.x, y=point.y)))
        comparisons_sampled["Surveyed Cross Section Elevations z"].append(point.z)
        comparisons_sampled["Section"].append(row['Section'])
        comparisons_sampled["River"].append(polygon.iloc[0].geometry.contains(point))
        comparisons_sampled["Distance Upstream [m]"].append(distance_upstream)
        comparisons_sampled["Offset [m]"].append(offset)
        
comparisons_sampled = pandas.DataFrame.from_dict(comparisons_sampled)

# Plot sampled elevations along entire sections
Plot the sampled elevations along individual cross-sections including the on-land portions as a sanity check.

In [ ]:
columns = ["Surveyed Cross Section Elevations z"]; columns.extend([f"{key} z" for key in dems])
for section in ordered_measured_sections:
    figure = comparisons_sampled[comparisons_sampled["Section"]==section][columns].plot(title=section)
    figure.figure.savefig(cache_path / folder / "measured_section_comparison" / f"{section}.png")
    matplotlib.pyplot.close()
    
    distance = comparisons_sampled[comparisons_sampled['Section']==section]['Distance Upstream [m]'].iloc[0]
    figure = comparisons_sampled[comparisons_sampled["Section"]==section][columns].plot(title=f"{distance}m distance upstream")
    figure.figure.savefig(cache_path / folder / "measured_section_comparison" / "distance_upstream" / f"{distance}m_upstream.png")
    matplotlib.pyplot.close()

In [ ]:
columns = ["Surveyed Cross Section Elevations z"]; columns.extend([f"{key} z" for key in dems])
for section in ordered_measured_sections:
    figure = comparisons_sampled[(comparisons_sampled["Section"]==section) & (comparisons_sampled["River"]==True)][columns].plot(title=section);
    figure.figure.savefig(cache_path / folder / "measured_section_comparison" / f"only_river_{section}.png")
    matplotlib.pyplot.close()
    
    distance = comparisons_sampled[comparisons_sampled['Section']==section]['Distance Upstream [m]'].iloc[0]
    figure = comparisons_sampled[(comparisons_sampled["Section"]==section) & (comparisons_sampled["River"]==True)][columns].plot(title=f"{distance}m distance upstream")
    figure.figure.savefig(cache_path / folder / "measured_section_comparison" / "distance_upstream" / f"only_river_{distance}m_upstream.png")
    matplotlib.pyplot.close()

## Plot elevation errors along each section
95% confidence interval plot for the errors along each section in the river. 

In [ ]:
for key in dems:
    comparisons_sampled[f"{key} Elevation Errors"] = comparisons_sampled[f"{key} z"] - comparisons_sampled["Surveyed Cross Section Elevations z"]

In [ ]:
comparisons_sampled.head(1)

In [ ]:
comparisons_for_plotting=comparisons_sampled.drop(
    columns=[f"{key} z" for key in dems]).drop(columns=["Surveyed Cross Section Elevations z"]).rename(
    columns=dict((f"{key} Elevation Errors", f"{key}") for key in dems)).melt(
    id_vars=["Section", "Distance Upstream [m]", "River", "Offset [m]"], var_name="Comparison", value_name="Riverbed Elevation Error [m]")

In [ ]:
comparisons_for_plotting.to_json(cache_path / folder / "measured_section_comparison" / "comparison_for_plotting.json")

In [ ]:
seaborn.set(rc={'figure.figsize':(11.7,6)})
figure = seaborn.lineplot(data=comparisons_for_plotting[comparisons_for_plotting["River"]], x="Distance Upstream [m]", y="Riverbed Elevation Error [m]", hue="Comparison")
figure.set(title='Section Depths Comparison 95% CI')
figure.text(-2150, 8.5, "a", fontsize=14)
figure.get_figure().savefig(cache_path / folder / "measured_section_comparison" / "section_depth_errors.png") 

In [ ]:
summary = comparisons_sampled[comparisons_sampled['River']].drop(
    columns=[f"{key} z" for key in dems]).set_index(
    keys=["Distance Upstream [m]", "Offset [m]"])
depth_summary = pandas.DataFrame([[summary.mean()[f"{key} Elevation Errors"] for key in dems],
                                  [summary.std()[f"{key} Elevation Errors"] for key in dems]],
                                  index = ["Mean Error", "Standard Deviation"], columns=[key for key in dems])
depth_summary

# Channel Area error
The total area of the channel at each section

In [ ]:
channel_depths = comparisons_sampled[comparisons_sampled['River']].drop(
    columns=[f"{key} Elevation Errors" for key in dems]).drop(columns=["River", "Section"]).set_index(
    keys=["Distance Upstream [m]", "Offset [m]"])
section_max_evaluation = channel_depths.groupby("Distance Upstream [m]")[["Surveyed Cross Section Elevations z"]].max()

In [ ]:
channel_depths.head(1)

In [ ]:
# Some hacky code to subtract the max elevation from each column in turn
channel_depths["Surveyed Cross Section Elevations"] = section_max_evaluation - channel_depths[["Surveyed Cross Section Elevations z"]]
for key in dems:
    channel_depths[key] = section_max_evaluation.rename(columns={"Surveyed Cross Section Elevations z": f"{key} z"}) - channel_depths[[f"{key} z"]]
channel_depths.head(1)

In [ ]:
channel_area = channel_depths.drop(
    columns=["Surveyed Cross Section Elevations z"]).drop(
    columns=[f"{key} z" for key in dems]).groupby("Distance Upstream [m]").sum()
channel_area.head(1)

In [ ]:
areas_for_plotting=channel_area.reset_index().melt(id_vars=["Distance Upstream [m]"], var_name="Source", value_name="Riverbed Area [m^2]")

In [ ]:
seaborn.set(rc={'figure.figsize':(11.7,8.27)})
figure = seaborn.lineplot(data=areas_for_plotting, x="Distance Upstream [m]", y="Riverbed Area [m^2]", hue="Source")
figure.set(title='Channel Area Comparison')
figure.get_figure().savefig(cache_path / folder / "measured_section_comparison" / "section_area_errors.png") 

# Exploring potential sources of error
## 1. Using measured versus estimate flow
The flow guage stations provide measured mean-annual flood flows. These can be used instead of the estimated ones across NZ. We can look at the impact of the different values by simply scaling the depths and then rerunning the generate hydrologically conditioned DEM steps.

#### Measured flow guage data
* [Te Kuha](https://www.lawa.org.nz/explore-data/west-coast-region/water-quantity/monitoring-sites/buller-rv-te-kuha/) - 3727.09 m³/s

#### Estimated flow data
* Te Kuha section NZREACH ID 12012463 - 4225.07 m³/s

#### Scaling to the different depths
In both cases the equation can be re-arragned to: h=(n/wS^b)^1/(1+a)*Q^1/(1+a) where 1/(1+a) differs in each case.
So depth can be scaled by (Qmeasured/Qold) ^ 1/(1+a). Where Qmeasured/Qold = 0.8821368640046201.
This means in each case:
* Uniform flow - 1/(1+a) = 0.6 -> x 0.9275163689031669
* Conceptual multivariate regression - 1/(1+a) = 0.573 -> x 0.9306622787286434
So depth can be scaled by (Qmeasured/Qold) ^ 1/(1+a)

## 2. Different thresholds
##### Currently
* narrow is 0.5 min to 0.75 max
* wide is 2 min to 5 max
Looking through the sampled sections it looks like the banks are higher than  5m generally (at least for bank full flow), and the water section could be less than 2m - try say 1m min, 10 max

## 3. Turn off the depth adjustment
#### Currently 
* My code for adjusting the depth estimate based on the banks doesn't seem to be quite right!
* It also makes the rivers deeper which is not desired based on a comparison to the sections
#### Future
Drope out this 'correction' as an unneeded complexity at this stage. 

## 4. Could add a proportion flowing through sediment
In braided rivers most or even all flow can be through the river gravels. 

#### Future
1. Could add a user specified 'proportion of flow in sediments' 
2. May just want to keep at it is as trying to make a 'hydrologically conditioned DEM' as opposed to the actual measured cross sections.

Questions: are their models where would want the actual bed elevation as they model the flow through the gravels - I think so - sediment transport models

# Explore depths

In [ ]:
river_depths = geopandas.read_file(cache_path / folder / "wide_river" / "river_bathymetry.geojson")
river_depths_all = geopandas.read_file(cache_path / folder / "wide_river" / "river_characteristics.geojson")

In [ ]:
river_depths_all.columns

In [ ]:
import matplotlib.pyplot
fig, ax = matplotlib.pyplot.subplots(ncols=1, nrows=1, figsize=(10, 5))
(river_depths_all[["slope_mean_0.05km", 'slope_mean_0.5km']]*1000).plot(ax=ax)
river_depths[["bed_elevation_Neal_et_al", "bed_elevation_Rupp_and_Smart"]].plot(ax=ax)

In [ ]:
river_depths_all[["min_z_centre_unimodal"]].plot()

In [ ]:
river_depths.head(0)

In [ ]:
river_depths_all[["valid", "valid_widths", "valid_threhold", "slope_mean_0.05km"]].plot()

In [ ]:
river_depths[["bed_elevation_Neal_et_al", "bed_elevation_Rupp_and_Smart", "bank_height"]].plot()

In [ ]:
river_depths[["width", "flat_widths_mean_0.05km"]].plot()

In [ ]:
river_depths[["depth_Neal_et_al", "flood_depth_Neal_et_al"]].plot()

In [ ]:
river_depths_all[["valid_threhold", "thresholds_mean_0.05km", "thresholds_mean_0.15km", "thresholds_mean_0.2km", "thresholds_mean_0.25km"]].plot() # "threshold", 

In [ ]:
pandas.concat({"Depth":(river_depths_all["min_z_centre_unimodal"] - river_depths["depth_Neal_et_al"]),
               "Flood Depth": (river_depths_all["min_z_centre_unimodal"] - river_depths["flood_depth_Neal_et_al"] + river_depths_all["thresholds_mean_0.05km"]),
               "Flood Depth 0.25": (river_depths_all["min_z_centre_unimodal"] - river_depths["flood_depth_Neal_et_al"] + river_depths_all["thresholds_mean_0.25km"])},axis=1).plot()

In [ ]:
(river_depths_all["min_z_centre_unimodal"] - river_depths["depth_Neal_et_al"] + river_depths_all["thresholds_mean_0.05km"]).plot()

In [ ]:
(river_depths_all["min_z_centre_unimodal"] - river_depths["flood_depth_Neal_et_al"] + river_depths_all["thresholds_mean_0.05km"]).plot()

In [ ]:
(river_depths_all["min_z_centre_unimodal"] - river_depths["flood_depth_Neal_et_al"] + river_depths_all["threshold"]).plot()